## LLM

In [2]:
# set path

import sys
sys.path.insert(0, '../src/')

import matplotlib.pyplot as plt
from tokenizer import TokenizerBPE
from data_handling import normalize_to_ascii, clean_text
from utils import saver
import numpy as np
import re

import os
import pickle as pkl
from tqdm.notebook import tqdm
import json

# disable gpu for testing purposes
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


## Standford

In [3]:
import json

with open('../corpus/train-v2.0.json', 'r', encoding='utf-8') as f:
    squad = json.load(f)

In [4]:
context_list = []

num_articles = len(squad['data'])
question_grouped = []
answer_grouped = []

for article in squad['data']:
    title = article['title']
    for paragraph in article['paragraphs']:
        question_grouped.append([])
        answer_grouped.append([])
        context_list.append(clean_text(normalize_to_ascii(paragraph['context'])))
        for qa in paragraph['qas']:
            if not qa['is_impossible']:
                
                question = qa['question']
                answer = qa['answers']

                question_grouped[-1].append(clean_text(normalize_to_ascii(question)))
                answer_grouped[-1].append(clean_text(normalize_to_ascii(answer[0]['text'])))


In [5]:
saver("../corpus/squad_raw", [context_list, question_grouped, answer_grouped])

In [5]:
def fused_sqa(story_list, question_grouped_list, answer_grouped_list):
    q ="<q>"
    a = "<a>"
    sos = "<s>"
    eos = "</s>"

    rcw = re.compile(r"\s+")

    corpus_list = []
    for story, question_list, answer_list in tqdm(list(zip(story_list, question_grouped_list, answer_grouped_list))):
        story = story.replace("\n", " ").replace("\r", " ")
        story = rcw.sub(" ", story).strip()
        sqa = [sos, normalize_to_ascii(story).lower()]
        for question, answer in zip(question_list, answer_list):
            question = question.replace("\n", " ").replace("\r", " ")
            question = rcw.sub(" ", question).strip()
            answer = answer.replace("\n", " ").replace("\r", " ")
            answer = rcw.sub(" ", answer).strip()
            sqa.append(q)
            sqa.append(normalize_to_ascii(question).lower())
            sqa.append(a)
            sqa.append(normalize_to_ascii(answer).lower())
        sqa.append(eos)
        corpus_list.append("".join(sqa))
        
    return corpus_list

In [17]:
def pad_corpus(corpus, max_length, tokenizer):
    pad_token = tokenizer.token_to_idx["<pad>"]
    padded_corpus = []
    for line in tqdm(corpus):
        if line.shape[1] < max_length:
            padding = tf.repeat(tf.constant([[pad_token]]), max_length - line.shape[1], axis=1)
            padded_line = tf.concat([line, padding], axis=1)
        else:
            padded_line = line[:, :max_length]
        padded_corpus.append(padded_line)
    return tf.concat(padded_corpus, axis=0)

In [22]:
corpus_encoded = pkl.load(open('../corpus/corpus_clean/corpus_squad_sqa_24k', 'rb'))
corpus_padded = pad_corpus(corpus_encoded, 768, tokenizer)
pkl.dump(corpus_padded, open('../corpus/corpus_clean/corpus_squad_sqa_24k_padded', 'wb'))

  0%|          | 0/19035 [00:00<?, ?it/s]

## 